In [1]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Pose.
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Initialize Video Capture
cap = cv2.VideoCapture(0)

# Variables to keep track of counting
step_count = 0
foot_lifted = {'left': False, 'right': False}

while cap.isOpened():
    success, image = cap.read()
    if not success:
        break

    # Convert the image to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image)

    # Convert back to BGR for OpenCV
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Draw the pose annotations on the image
    if results.pose_landmarks:
        mp.solutions.drawing_utils.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        try:
            # Extract landmarks for both heels
            left_heel = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HEEL]
            right_heel = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HEEL]

            # Define a threshold for when a foot is considered "lifted"
            lifted_threshold = 0.6

            # Check and update for the left foot
            if left_heel.visibility > 0.5:
                if not foot_lifted['left'] and left_heel.y < lifted_threshold:
                    foot_lifted['left'] = True
                elif foot_lifted['left'] and left_heel.y > lifted_threshold:
                    foot_lifted['left'] = False
                    step_count += 1

            # Check and update for the right foot
            if right_heel.visibility > 0.5:
                if not foot_lifted['right'] and right_heel.y < lifted_threshold:
                    foot_lifted['right'] = True
                elif foot_lifted['right'] and right_heel.y > lifted_threshold:
                    foot_lifted['right'] = False
                    step_count += 1

        except IndexError:
            pass

    # Display the step count
    cv2.putText(image, f'Steps: {step_count}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    # Show the image
    cv2.imshow('Run in Place Counter', image)

    # exit the program by pressing "q" on the keyboard
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


2023-11-29 00:50:39.522260: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
